# Tenzin Tashi
## CSc 301
### Assignment 4

<center> <h3> Math </h3> </center>

$$ \int_a^b f(x) dx $$ 
<br>
<strong> <center>Simpson's rule</center> </strong><br>
$$ \int_a^b f(x) dx \approx \frac{h}{3} \Bigg[ f(a) + 4 f(\frac{a+b}{2}) + f(b) \Bigg] - \frac{(b - a)*h^4}{180}f''''(\xi)$$
<br>
$$ S(a,b) = \frac{h}{3} \Bigg[ f(a) + 4 f(\frac{a+b}{2}) + f(b) \Bigg] - \frac{(b - a)*h^4}{180}f''''(\xi)$$
<br>
<strong> <center>Composite Simpson's rule</center> </strong>
<br>
$$ Let \ c = b - a $$
<br>
$$ S_0 (a, b) = S(a, b)$$
<br>
$$ S_1 (a, b) = S\Big(a, a + \frac{c}{2}\Big) + S\Big(a + \frac{c}{2}, b\Big)$$
<br>
$$ S_2 (a, b) = S\Big(a, a + \frac{c}{4}\Big) + S\Big(a + \frac{c}{4}, a + \frac{c}{2}\Big) + S\Big(a + \frac{c}{2}, a + \frac{3c}{4}\Big) + S\Big(a + \frac{3c}{4}, b\Big)$$
<br>
<b>We can estimate the error in $$S_n (a, b) = |S_n (a,b) - S_{n-1} (a, b) |$$</b>

In [1]:
# import the libery
import numpy as np
from scipy.misc import derivative
from scipy import integrate

In [2]:
def simpson_rule(f, a, b):
    '''Approximate the integral of f(x) from a to b by Simpson's rule.

    Simpson's rule approximates the integral \int_a^b f(x) dx by the sum,
    as shown above in the Math section, where h = \frac{b-a}{2}.

    Parameters
    ----------
    f : function
        Vectorized function of a single variable
    a , b : numbers
        Interval of integration [a,b]

    Returns
    -------
    float
        Approximation of the integral of f(x) from a to b using
        Simpson's rule.
        
    '''
    h = (b - a)/2
    return h*(f(a) + 4*f((a + b)/2) +f(b))/3 - (h**4)*(derivative(f,2.0,dx=4))/180
    

In [3]:
def composite_simpson(f, a, b, EB):
    '''Approximate the integral of f(x) from a to b within the error 
    bound of EB using Simpson's rule.

    The math for Composite Simpson's rule formula is shown above.

    Parameters
    ----------
    f : function
        Vectorized function of a single variable
    a , b : numbers
        Interval of integration [a,b]
    EB : number
        Value for the error bound

    Returns
    -------
    float, Int
        Approximation of the integral of f(x) from a to b which is 
        within the error bound using Simpson's rule, and the number 
        if intervals needed for it.
        
    '''
    N = 0 # start the number of interval from 0
    S = list() # a list that hold the integral value for increasing interval
    c = b - a
    interval = int
    flag = True
    while flag: # loop untill we are in the error bound
        Si = 0 # simpson's integral for ith interval
        interval = 2**N
        s = a # start of the interval
        e = a + c/interval # end of the interval
        for _ in range(interval):
            Si += simpson_rule(f,s, s + c/interval)
            s = e
            e += c/interval
        S.append(Si)
        if N != 0:
            if np.abs(S[N] - S[N-1]) < EB:
                return S.pop(), N
        N += 1

In [4]:
def adaptive_quadrature(f, a, b, tol):
    '''
    Evaluates the integral of f(x) on [a,b].

    Parameters
    ----------
        f         - function to integrate
        a, b      - left and right endpoints of interval of integration
        tol       - desired upper bound on allowable error

    Returns
    -------
        float     - the value of the integral
        Int       - the number of intervals
        
    '''
    
    N = 0
    APP = 0
    i = 1
    TOL = [0] * 2000
    TOL.insert(i, 10 * tol) # not sure
    ai = [0] * 2000
    ai.insert(i, a)
    h = [0] * 2000
    h.insert(i, (b-a)/2)
    FA = [0]* 2000
    FA.insert(i, f(a))
    FC = [0] * 2000
    FC.insert(i, f(a + h[i]))
    FB = [0] * 2000
    FB.insert(i, f(b))
    S = [0] * 2000
    S.insert(i, h[i]*(FA[i] + 4*FC[i] + FB[i]) / 3)

    L = [0] * 2000
    L.append(1)

    # Step 2: while loop steps  3-5
    while (i > 0):
        # Step 3
        FD = f(ai[i] + (h[i]/2))
        FE = f(ai[i] + (3*h[i])/2)
        S1 = h[i]*(FA[i] + 4*FD + FC[i]) / 6
        S2 = h[i]*(FC[i] + 4*FE + FB[i]) / 6
        v1 = ai[i]
        v2 = FA[i]
        v3 = FC[i]
        v4 = FB[i]
        v5 = h[i]
        v6 = TOL[i]
        v7 = S[i]
        v8 = L[i]

        # Step 4
        i -= 1

        # Step 5
        if (abs(S1 + S2 - v7) < v6):
            APP = APP + (S1 + S2)
        else: 
            i += 1 # Data for right-half subinterval
            ai[i] = v1 + v5
            FA[i] = v3
            FC[i] = FE
            FB[i] = v4
            h[i] = v5/2
            TOL[i] = v6/2
            S[i] = S2
            L[i] = v8 + 1

            i += 1 # Data for left-half subinterval
            ai[i] = v1
            FA[i] = v2
            FC[i] = FD
            FB[i] = v3
            h[i] = h[i-1]
            TOL[i] = TOL[i-1]
            S[i] = S1
            L[i] = L[i-1]
            N += 1

    # Step 6: OUTPUT (APP) 
    return APP, N

In [5]:
if __name__ == '__main__':
    f = lambda x: (100/x**2)*np.sin(10/x)
    exact_value = integrate.quad(f, 1, 3)[0]
    simp, N = composite_simpson(f, 1, 3, 10**(-8))
    adap, M = adaptive_quadrature(f, 1, 3, 10**(-8))
    abs_err_simp = np.abs(exact_value - simp)
    abs_err_adap = np.abs(exact_value - adap)
    print(f'{"Method":30}{"Integral Value":20}{"Exact Value":25}{"Abs Error":25}{"Steps":5}')
    print(f'{"Composite Simpsons Rule:":25} {simp:20} {exact_value:20} {abs_err_simp:24} {2**N:11}')
    print(f'{"Adaptive quadrature :":26} {adap:10} {exact_value:20}  {abs_err_adap:24} {M:10}')
    

Method                        Integral Value      Exact Value              Abs Error                Steps
Composite Simpsons Rule:   -1.4260247557378427   -1.426024756346262     6.08419314929165e-10        1024
Adaptive quadrature :      -1.4260247550720377   -1.426024756346262     1.274224281644365e-09        262


##### <center><h3>Discussion</h3></center>

<p>Integral of a function can be found by different methods. In this assignment we have looked into the two methods namely, composite simpson's rule and adaptive quadrature.<br>
    <strong>Composite simpson's rule</strong> divides the area of the function into equal number of smaller units. This smaller units are called the interval which is calculated as shown in the above math section as 2^n, where n is the interval. After dividing it into smaller units we applied the simpson's rule to every units. The summation of all this smaller units will give us the integral of the original function.<br>
    <strong>Adaptive quadrature </strong> is an extension of the composite simpson's rule. In this method we divide the intevals into futher sub-intervals <strong>(where needed)</strong>, due to which it increase the accuracy of this method.</p>
    <br>
    <strong>NOTE:</strong> To get the result with same error bound we can see in the above output that composite simpson's rule makes a sub-interval of 1024 where as adaptive quadrature only takes 262 intervals.